In [1]:
import os
import glob
import pandas as pd
def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }

    dirs = glob.glob(os.path.join(dn,"*"))
    for d in dirs:
        pat = os.path.join(d,"*.txt")
        fs = glob.glob(pat)
    #     print(fs)
        for fp in fs:
            with open(fp,"r",encoding="utf-8") as f:
                news = f.read()
            ans = os.path.split(d)[-1]
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data,columns=["news","ans"])
    return df

In [2]:
train_df = get_data("chinese_news_trans/")
test_df = get_data("chinese_news_test/")
test_df

,news,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [3]:
{c:i for i , c in enumerate(set(train_df["ans"]))}

{'政治': 0,
 '醫藥': 1,
 '交通': 2,
 '經濟': 3,
 '軍事': 4,
 '環境': 5,
 '計算機': 6,
 '教育': 7,
 '體育': 8,
 '藝術': 9}

In [4]:
#unique value_counts
kind = train_df["ans"].unique() 
#enumerate
trans = {k:i for i,k in enumerate(kind)}
reverse_trans = {i:k for i,k in enumerate(kind)}
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)

In [5]:
import os
from urllib.request import urlretrieve 
DICT_PATH = "dict.txt.big"
if not os.path.exists(DICT_PATH):
    print("字典不存在")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url,DICT_PATH)

In [6]:
import jieba
jieba.set_dictionary(DICT_PATH)
def news_cut(news):
    s = " ".join(jieba.cut(news))
    return s.replace("\r","").replace("\n","")
x_train_raw = train_df["news"].apply(news_cut)
x_test_raw = test_df["news"].apply(news_cut)

Building prefix dict from D:\Python_Summarize\PycharmProjects\Ace_Deep_Learning\dict.txt.big ...
Loading model from cache C:\Users\Ace\AppData\Local\Temp\jieba.ua3a4bf2f10aef43ed7a8502ff58d96e3.cache
Loading model cost 1.134 seconds.
Prefix dict has been built successfully.


In [7]:
x_test_raw

0      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
1      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
2      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
3      三趟 火車 停開   乘客 可 全額 退票    瀏覽 次數 ： 1180         ...
4      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
                             ...                        
96     最 優秀 選手 無緣 亞運會 健美 賽     　            健美 在 亞洲 運...
97        各國 記者 眼中 的 羽毛球 世錦賽      -------------------...
98      友好 運動會 第五天   東道主 選手 大顯神威  2001 年 09 月 03 日 02...
99     不靠 技術 比 運氣   　 　 第二屆 奧運會 在 巴黎 舉行 ， 同時 這裡 也 正在 ...
100    帆   板   運   動   簡   介 （ 二 ）   　 　 我國 在 79 年 由 ...
Name: news, Length: 101, dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(x_train_raw)
x_test = vec.transform(x_test_raw)


In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [10]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre,y_test)

1.0

In [11]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test,pre)
col = ["{}(預測)".format(k) for k in kind]
row = ["{}(答案)".format(k) for k in kind]
pd.DataFrame(mat,columns=col,index=row)

,交通(預測),政治(預測),教育(預測),環境(預測),經濟(預測),藝術(預測),計算機(預測),軍事(預測),醫藥(預測),體育(預測)
交通(答案),10,0,0,0,0,0,0,0,0,0
政治(答案),0,11,0,0,0,0,0,0,0,0
教育(答案),0,0,10,0,0,0,0,0,0,0
環境(答案),0,0,0,10,0,0,0,0,0,0
經濟(答案),0,0,0,0,10,0,0,0,0,0
藝術(答案),0,0,0,0,0,10,0,0,0,0
計算機(答案),0,0,0,0,0,0,10,0,0,0
軍事(答案),0,0,0,0,0,0,0,10,0,0
醫藥(答案),0,0,0,0,0,0,0,0,10,0
體育(答案),0,0,0,0,0,0,0,0,0,10


In [15]:
n = input("請輸入新聞:")
test = vec.transform([news_cut(n)])
pre = clf.predict(test)[0]
print("應該是:",reverse_trans[pre])
proba = clf.predict_proba(test)[0]
proba = list(enumerate(proba))
# def test(x):
#     return x[1]
for i,prob in sorted(proba,key=lambda x:x[1],reverse=True):
    print(reverse_trans[i],":",prob*100,"%")
    

請輸入新聞:國內昨又新增7例確診感染新冠肺炎個案，其中，比利時籍風電工程師未列入本土感染或境外移入病例，中央流行疫情指揮中心指揮官陳時中稱目前還無法判定，需再釐清感染源，但前疾管局長、抗煞專家蘇益仁認為該名工程師抗體IgG陽性且在台待近3個月，直言「可代表是在本土感染」，意味社區內有無症狀患者！
應該是: 醫藥
醫藥 : 99.9992764188479 %
計算機 : 0.0005225636648301281 %
交通 : 0.00012394772051973072 %
軍事 : 5.0452253915962506e-05 %
經濟 : 2.205799933094503e-05 %
藝術 : 4.437112656501239e-06 %
環境 : 1.1717169049195181e-07 %
體育 : 5.105083926648924e-09 %
教育 : 1.1599798842467697e-10 %
政治 : 7.040352778706514e-12 %
